# Download SWOT swaths that intersect with a specific region from the AVISO THREDDS Server

### Here we'll use some scripts to download SWOT data in a specific region from AVISO THREDDS. 
### Downloading from THREDDS using HTTP GET requests is much faster than sftp (see example notebook 002...) and allows for parallel downloads.

- This notebook uses scripts from <code>src/THREDDS_download_swaths.py</code>. 
- Note that we are using the <code>reqiests.get()</code> method to directly pull swaths from the [AVISO THREDDS](https://tds-odatis.aviso.altimetry.fr/thredds/catalog/catalog.html) data catalog using an <code>HTTP GET</code> request, and as such we need to download each swath in it's entirety before subsetting. This is a little inefficient but at least it's faster than my previous sftp method...
- You can just use my login information, provided here, to access AVISO
- NOTE: As an example, I wrote this script to save L3 SWOT swaths to a folder in the base directory named "<code>SWOT_L3</code>", which has been added to the <code>.gitignore</code> file. You should change the save path to wherever is most convienent for you. If you save your files to a different directory on the main path remember to add the save directory to <code>.gitignore</code> to avoid problems with large files.


### Before you begin:
- Make sure you've downloaded the orbit shape files to tell the script where the specific tracks are. You need two shape files for the open ocean's calval and science phases, <code>sph_calval_nadir.zip</code> and  <code>sph_science_nadir.zip</code>, which you can download the here [here](https://www.aviso.altimetry.fr/en/missions/current-missions/swot/orbit/science-orbit.html). 

In [31]:
"""
Download SWOT swaths in a specific bounding box.

Dependencies:
paramiko
numpy
xarray

Author: Tatsu Monkman
Date: 2025-06-24
"""
import os
import time

# Add the path to the Tatsu's swot download library
import sys
sys.path.append('../src/')
import THREDDS_download_swaths


In [32]:
os.getcwd()

'/scratch/projects/shaferlab/tatsu/NYU_SWOT_project/SWOT-data-analysis/examples'

## Define lat/lon bounds for the region you are interested in

In [4]:
# Define bounding domain, here I'm downloading some swaths 
# over the Kuroshio east of Japan.

# Rough East of Japan domain (Kuroshio-ish)
kuroshio_sw_corner = [140,15]
kuroshio_ne_corner = [170,40]

# NOTE: I am specifically recording the Northern and Southern edges
# of the bounding box here for later use when I subset the data.
# I find it helpful to add them to a new list to keep everything 
# organized.
kuroshio_lat_lims = [kuroshio_sw_corner[1],kuroshio_ne_corner[1]]

## Specify the cycles you want to download

Key points:
- SWOT data is organized by orbit "cycle" on AVISO. Each numbered cycle corresponds to one full traversal of the individual swaths that make up a complete orbital cycle. Cycles numbered 474-578 correspond to 1-day repeat orbits made during the Cal/Val phase, while cycles numbered 001-016+ correspond to orbits made during the science phase. Note you need to keep the leading zeros on the labels for the low-numbered cycles since the filenames use standardized lengths.
- Each cycle is subdivided into individual "passes" which correspond to one North-South or South-North path of the SWOT satellite as it moves along its orbit. Each pass is given an identification number, which can be used to find specific swaths you are interested in. The orbital shape files in  <code>../orbit_data/</code> specify the pass identification number using the variable "PASS_ID".
- Pass IDs correspond to the order in which each pass occured during each cycle. For example, "Pass 1" in the science phase is the first pass of a given science cycle, and "Pass 1" in the Cal/Val phase is the first pass in a given Cal/Val cycle.
- Every "pass" is repeated for every "cycle". For example, if you want to find all the SWOT data in the California region you would first look at which passes travel through the region, then download those specific passes for all of the cycles you are interested in.

In [52]:
"""
Below is some code to specify the passes and cycles to download.

Here we are downloading passes in the Kuroshio from all of the Cal/Val cycles,
i.e. cycles 474-578, and the first Science cycle, cycle 001.

"""

# EXAMPLE: Cycles 001 - 016 are for the 21-day science phase
# cycles = [str(c_num).zfill(3) for c_num in range(1,17)]

# EXAMPLE: Cycles 474 - 578 are from the 1-day repeat Cal/Val phase
# cycles = [str(c_num).zfill(3) for c_num in range(474,578)]

# Here I'm just pulling the first 10 cycles from the 1-day repeat Cal/Val phase 
# and the first cycle of the science phase.
cycles = ["001"] + [str(c_num).zfill(3) for c_num in range(474,484)]


In [53]:
cycles

['001', '474', '475', '476', '477', '478', '479', '480', '481', '482', '483']

## Specify the path to the orbit shapefile

You need to specify the path to the orbit shapfiles so my scripts can figure out which swaths you want

In [54]:
orbit_file_path = "../orbit_data"

## Specify the path to the data on the AVISO THREDDS server

Next we want to specify where to look for the data on the AVISO ftp server. Here are some example paths for a) the L2 PID0 Unsmoothed data and b) the L3 v2.0.1 Unsmoothed data:

```python
    # Paths for L2 PID0 Unsmoothed data
    remote_path="https://tds-odatis.aviso.altimetry.fr/thredds/catalog/dataset-l2-swot-karin-lr-ssh-validated/PID0/Unsmoothed"
    # Paths for L3 v2.01 Unsmoothed data
    remote_path="https://tds-odatis.aviso.altimetry.fr/thredds/catalog/dataset-l3-swot-karin-nadir-validated/l3_lr_ssh/v2_0_1/Unsmoothed"    
```

If you want to poke around on the AVISO THREDDS server to find different data releases, you can access it here:

```unix
https://tds-odatis.aviso.altimetry.fr/thredds/catalog/catalog.html
```

I'm looking for version 2.0.1 release of the SWOT L3 Unsmoothed data, so I've specified the path to this specific release on the THREDDS server:

In [36]:
remote_path="https://tds-odatis.aviso.altimetry.fr/thredds/catalog/dataset-l3-swot-karin-nadir-validated/l3_lr_ssh/v2_0_1/Unsmoothed"

## Specify the remote path to access the data and the local_path you want to save it to on your machine
I'm saving the data in this example to my scratch directory on Greene.

In [55]:
save_path = "~/scratch/SWOT_L3_example/Unsmoothed"

# Download the data

The script below downloads the data for the given passes and cycles you are interested in. Note I've instructed the script to generate an extra log file (the <code>skipped_swaths.txt</code> file) to record when there's a problem and the script fails to download the data.

Good luck!

In [50]:
from importlib import reload

THREDDS_download_swaths = reload(THREDDS_download_swaths)

ERROR! Session/line number was not unique in database. History logging moved to new session 1522


In [51]:
THREDDS_download_swaths.run_download(kuroshio_sw_corner, kuroshio_ne_corner, cycles, remote_path, save_path, orbit_file_path, ssh_kwargs)



Found 58 passes in 1 cycles for the science phase.
Found 20 passes in 10 cycles for the calval phase
Fetching catalog for cycle 001...
✔ File already exists, skipping: ~/scratch/SWOT_L3_example/Unsmoothed/cycle_001/SWOT_L3_LR_SSH_Unsmoothed_001_155_20230726T173637_20230726T182804_v2.0.1.nc
✔ File already exists, skipping: ~/scratch/SWOT_L3_example/Unsmoothed/cycle_001/cycle_001/SWOT_L3_LR_SSH_Unsmoothed_001_157_20230726T191931_20230726T201056_v2.0.1.nc


SWOT_L3_LR_SSH_Unsmoothed_001_170_20230727T062820_20230727T071946_v2.0.1.nc:   1%|          | 7.34M/681M [00:06<09:34, 1.17MB/s]


KeyboardInterrupt: 